# Import package for data processing

In [108]:
import json
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from functools import reduce
from collections import Counter
from nltk.corpus import stopwords
from nltk.corpus import brown
from string import punctuation, digits
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
wnl = WordNetLemmatizer()
import numpy as np

np.random.seed(2018)
nltk.download('wordnet')

with open("camera_reviews_full.json", 'r') as f:
    js = json.load(f)
    f.close()
gensim.__version__

[nltk_data] Downloading package wordnet to /Users/hungting-
[nltk_data]     yi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'3.1.0'

## To run the LDA model for this script, we need to prepare Gensim 3.1.0. If using the version other than 3.1.0, it'll produce NaN and same words in each topic.
    downgrading Gensim in Mac terminal:
       $ pip3 install -Iv gensim==3.1.0

# Exclude punctuations, stopwords, alphabets and digits. And lemmatizing the words.

In [109]:
stopwords_set = set(stopwords.words('english'))
brown_set = set(brown.words())
characterstoclean = r'?!,:,/\"-+=@#$%^&*()><{}[]|' + r"'"
punc = punctuation

In [110]:
def get_pos( word ):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0]
def get_lemmatize_list(post_list):
    words = []
    for word in post_list.split(' '):
        word = word.lower()
        if word in brown_set and len(word) > 2 and word not in stopwords_set and word not in characterstoclean and word not in punc and not word.isdigit() and word.isalpha():
            words.append(word)
    list_temp=[]
    for word in words: 
        word = wnl.lemmatize( word, get_pos(word) )
        list_temp.append(word)
    return list_temp

In [111]:
js_lda = js.copy()
for i in js_lda.keys():
    for j in js_lda[i].keys(): 
        if j == "review":
            js_lda[i][j] = get_lemmatize_list(js_lda[i][j]) 

# Prepare packages for LDA model

# Create two lists which store verified and unverified customers' reviews separately for running LDA model.

In [112]:
lda_list_verified=[]
lda_list_unverified=[]
for i in js_lda.keys():
    if js_lda[i]["author_attributes"] == []:
        lda_list_unverified.append(js_lda[i]["review"])
    elif "Verified" in js_lda[i]["author_attributes"][0]:
        lda_list_verified.append(js_lda[i]["review"])
    else:
        lda_list_unverified.append(js_lda[i]["review"])

In [113]:
len(lda_list_unverified), len(lda_list_verified)

(31, 3459)

In [114]:
dict_unverified = corpora.Dictionary(lda_list_unverified)
print(dict_unverified)
doc_term_matrix_unverified = [dict_unverified.doc2bow(doc) for doc in lda_list_unverified]
# print(doc_term_matrix_unverified)

Dictionary(287 unique tokens: ['great', 'starter', 'camera', 'come', 'everything']...)


In [128]:
Lda_unverified = gensim.models.ldamodel.LdaModel
lda_model_unverified = Lda_unverified(doc_term_matrix_unverified, num_topics=3, id2word = dict_unverified, passes=50)

In [129]:
for idx_unverified, topic_unverified in lda_model_unverified.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_unverified, topic_unverified))

Topic: 0 
Words: 0.042*"camera" + 0.024*"great" + 0.024*"lens" + 0.020*"get" + 0.020*"use" + 0.018*"easy" + 0.018*"well" + 0.018*"quality" + 0.017*"take" + 0.016*"canon"
Topic: 1 
Words: 0.024*"take" + 0.022*"camera" + 0.022*"gift" + 0.016*"picture" + 0.015*"use" + 0.015*"want" + 0.015*"upgrade" + 0.015*"advantage" + 0.015*"card" + 0.009*"good"
Topic: 2 
Words: 0.039*"camera" + 0.036*"take" + 0.022*"picture" + 0.018*"love" + 0.018*"use" + 0.018*"want" + 0.015*"quality" + 0.015*"get" + 0.015*"come" + 0.011*"buy"


In [131]:
unverifie_topics = lda_model_unverified.get_document_topics(doc_term_matrix_unverified, per_word_topics=True)
review = 1
for doc_topics, review_word_topics, phi_values in unverifie_topics:
    print('review: {} \n'.format(review))
    print('review topics:', doc_topics)
    max = 0
    for i in range(len(doc_topics)):
        if max < doc_topics[i][1]:
            max = doc_topics[i][1]
            doc_topics_hightest = doc_topics[i][0]
            
    print('review topics(highest probs): Topic ', doc_topics_hightest)
    print("words' topics:", review_word_topics)
    print('words related to the topics:\n', lda_model_unverified.print_topics(-1)[doc_topics_hightest][1])
    print('-------------- \n')
    review += 1

review: 1 

review topics: [(0, 0.9940672814546186)]
review topics(highest probs): Topic  0
words' topics: [(0, [0]), (1, [0]), (2, [0]), (3, [0]), (4, [0]), (5, [0]), (6, [0]), (7, [0]), (8, [0]), (9, [0]), (10, [0]), (11, [0]), (12, [0]), (13, [0]), (14, [0]), (15, [0]), (16, [0]), (17, [0]), (18, [0]), (19, [0]), (20, [0]), (21, [0]), (22, [0]), (23, [0]), (24, [0]), (25, [0]), (26, [0]), (27, [0]), (28, [0]), (29, [0]), (30, [0]), (31, [0]), (32, [0]), (33, [0]), (34, [0]), (35, [0]), (36, [0]), (37, [0]), (38, [0]), (39, [0]), (40, [0]), (41, [0]), (42, [0]), (43, [0]), (44, [0]), (45, [0]), (46, [0]), (47, [0]), (48, [0]), (49, [0]), (50, [0]), (51, [0]), (52, [0]), (53, [0]), (54, [0]), (55, [0]), (56, [0]), (57, [0]), (58, [0]), (59, [0]), (60, [0]), (61, [0]), (62, [0]), (63, [0]), (64, [0]), (65, [0]), (66, [0]), (67, [0]), (68, [0]), (69, [0]), (70, [0]), (71, [0]), (72, [0]), (73, [0]), (74, [0]), (75, [0]), (76, [0]), (77, [0]), (78, [0]), (79, [0]), (80, [0]), (81, [0]), 

In [42]:
dict_verified = corpora.Dictionary(lda_list_verified)
print(dict_verified)
doc_term_matrix_verified = [dict_verified.doc2bow(doc) for doc in lda_list_verified]
# print(doc_term_matrix_verified)

Dictionary(1775 unique tokens: ['camera', 'full', 'feature', 'entry', 'level']...)


In [43]:
Lda_verified = gensim.models.ldamodel.LdaModel
lda_model_verified = Lda_verified(doc_term_matrix_verified, num_topics=3, id2word = dict_verified, passes=50)

In [49]:
for idx_verified, topic_verified in lda_model_verified.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Topic: 0 
Words: 0.055*"camera" + 0.035*"love" + 0.033*"buy" + 0.027*"like" + 0.024*"canon" + 0.018*"daughter" + 0.017*"get" + 0.016*"one" + 0.016*"purchase" + 0.015*"use"
Topic: 1 
Words: 0.079*"camera" + 0.078*"great" + 0.073*"take" + 0.051*"picture" + 0.043*"love" + 0.035*"easy" + 0.032*"use" + 0.020*"learn" + 0.018*"get" + 0.016*"buy"
Topic: 2 
Words: 0.070*"camera" + 0.069*"great" + 0.050*"easy" + 0.044*"good" + 0.032*"use" + 0.026*"quality" + 0.026*"recommend" + 0.025*"lens" + 0.020*"price" + 0.020*"would"


In [132]:
verified_topics = lda_model_verified.get_document_topics(doc_term_matrix_verified, per_word_topics=True)
review = 1
for doc_topics, review_word_topics, phi_values in verified_topics:
    print('review: {} \n'.format(review))
    print('review topics:', doc_topics)
    max = 0
    for i in range(len(doc_topics)):
        if max < doc_topics[i][1]:
            max = doc_topics[i][1]
            doc_topics_hightest = doc_topics[i][0]

    print('review topics(highest probs): Topic ', doc_topics_hightest)
    print('words related to the topics:\n', lda_model_verified.print_topics(-1)[doc_topics_hightest][1])
    print("words' topics:", review_word_topics)
    print('-------------- \n')
    review += 1

review: 1 

review topics: [(0, 0.31260237369986765), (2, 0.6857387413052709)]
review topics(highest probs): Topic  2
words related to the topics:
 0.070*"camera" + 0.069*"great" + 0.050*"easy" + 0.044*"good" + 0.032*"use" + 0.026*"quality" + 0.026*"recommend" + 0.025*"lens" + 0.020*"price" + 0.020*"would"
words' topics: [(0, [2, 0]), (1, [2, 0]), (2, [2, 0]), (3, [2]), (4, [2]), (5, [2, 0]), (6, [0, 2]), (7, [0, 2]), (8, [0, 2]), (9, [0]), (10, [2, 0]), (11, [0, 2]), (12, [0, 2]), (13, [0, 2]), (14, [0, 2]), (15, [0, 2]), (16, [0, 2]), (17, [0, 2]), (18, [0]), (19, [2, 0]), (20, [2, 0]), (21, [2, 0]), (22, [2, 0]), (23, [2, 0]), (24, [0, 2]), (25, [0, 2]), (26, [2, 0]), (27, [2, 0]), (28, [0]), (29, [2, 0]), (30, [2, 0]), (31, [2, 0]), (32, [2]), (33, [0]), (34, [2, 0]), (35, [2, 0]), (36, [2, 0]), (37, [2, 0]), (38, [2, 0]), (39, [2, 0]), (40, [2]), (41, [2, 0]), (42, [0, 2]), (43, [2, 0]), (44, [2, 0]), (45, [2]), (46, [2, 0]), (47, [2, 0]), (48, [2]), (49, [2, 0]), (50, [2, 0]), (5

words related to the topics:
 0.055*"camera" + 0.035*"love" + 0.033*"buy" + 0.027*"like" + 0.024*"canon" + 0.018*"daughter" + 0.017*"get" + 0.016*"one" + 0.016*"purchase" + 0.015*"use"
words' topics: [(0, [0, 1, 2]), (21, [0, 2]), (106, [0, 1]), (113, [0, 1]), (153, [0, 1, 2]), (168, [0]), (176, [0]), (178, [1, 2, 0]), (203, [0, 1]), (269, [0, 1]), (274, [0, 2]), (329, [2, 1]), (350, [0, 2]), (384, [0, 1]), (400, [0]), (402, [0]), (470, [0, 1, 2]), (491, [0, 1]), (546, [0, 1]), (565, [2, 0]), (575, [0, 1]), (580, [0]), (606, [0, 1]), (611, [0]), (648, [2, 1, 0]), (727, [0, 2]), (764, [0, 1, 2]), (765, [1, 0, 2]), (766, [0, 2, 1]), (767, [0]), (768, [0]), (769, [1, 0])]
-------------- 

review: 46 

review topics: [(0, 0.5714369825981251), (1, 0.19792975041609134), (2, 0.23063326698578354)]
review topics(highest probs): Topic  0
words related to the topics:
 0.055*"camera" + 0.035*"love" + 0.033*"buy" + 0.027*"like" + 0.024*"canon" + 0.018*"daughter" + 0.017*"get" + 0.016*"one" + 0.016*

review: 79 

review topics: [(0, 0.2695635948778818), (1, 0.4175808329077977), (2, 0.31285557221432053)]
review topics(highest probs): Topic  1
words related to the topics:
 0.079*"camera" + 0.078*"great" + 0.073*"take" + 0.051*"picture" + 0.043*"love" + 0.035*"easy" + 0.032*"use" + 0.020*"learn" + 0.018*"get" + 0.016*"buy"
words' topics: [(0, [1, 2, 0]), (2, [1, 2, 0]), (29, [1, 2, 0]), (33, [0]), (54, [2, 0]), (60, [2, 0, 1]), (62, [2, 1]), (68, [1, 0, 2]), (78, [2, 1]), (87, [1, 2]), (178, [1, 2]), (203, [1, 0]), (266, [1, 0]), (269, [0, 1]), (367, [1, 0, 2]), (457, [2, 1, 0]), (615, [1, 2, 0]), (756, [2, 0]), (811, [2, 0]), (873, [1]), (886, [0, 2, 1]), (887, [0, 2, 1])]
-------------- 

review: 80 

review topics: [(0, 0.21730879234711287), (1, 0.5042969308765204), (2, 0.2783942767763667)]
review topics(highest probs): Topic  1
words related to the topics:
 0.079*"camera" + 0.078*"great" + 0.073*"take" + 0.051*"picture" + 0.043*"love" + 0.035*"easy" + 0.032*"use" + 0.020*"learn" +

review: 123 

review topics: [(0, 0.8901359757084703), (1, 0.09261785757555248), (2, 0.017246166715977396)]
review topics(highest probs): Topic  0
words related to the topics:
 0.055*"camera" + 0.035*"love" + 0.033*"buy" + 0.027*"like" + 0.024*"canon" + 0.018*"daughter" + 0.017*"get" + 0.016*"one" + 0.016*"purchase" + 0.015*"use"
words' topics: [(2, [0, 1]), (43, [0]), (100, [0]), (119, [0, 1]), (125, [0]), (168, [0]), (178, [1, 0]), (268, [0]), (287, [0]), (351, [0, 1]), (389, [0, 1]), (643, [0, 1]), (727, [0]), (826, [0]), (1009, [0]), (1010, [0]), (1011, [0]), (1012, [0])]
-------------- 

review: 124 

review topics: [(0, 0.326215666890675), (1, 0.46998861424570404), (2, 0.20379571886362094)]
review topics(highest probs): Topic  1
words related to the topics:
 0.079*"camera" + 0.078*"great" + 0.073*"take" + 0.051*"picture" + 0.043*"love" + 0.035*"easy" + 0.032*"use" + 0.020*"learn" + 0.018*"get" + 0.016*"buy"
words' topics: [(0, [1, 0, 2]), (8, [0, 1]), (19, [0, 2]), (29, [1, 2, 0]

review: 165 

review topics: [(0, 0.6052518209885002), (1, 0.3729102062676145), (2, 0.0218379727438853)]
review topics(highest probs): Topic  0
words related to the topics:
 0.055*"camera" + 0.035*"love" + 0.033*"buy" + 0.027*"like" + 0.024*"canon" + 0.018*"daughter" + 0.017*"get" + 0.016*"one" + 0.016*"purchase" + 0.015*"use"
words' topics: [(0, [0, 1]), (93, [0, 1]), (168, [0]), (178, [1]), (189, [1, 0]), (197, [1, 0]), (200, [1, 0]), (211, [0]), (299, [0, 1, 2]), (434, [1]), (487, [0]), (1091, [0, 1]), (1092, [0, 1]), (1093, [0, 1, 2]), (1094, [1, 0]), (1095, [0, 1])]
-------------- 

review: 166 

review topics: [(0, 0.7496886141445275), (1, 0.23241123472317382), (2, 0.017900151132298668)]
review topics(highest probs): Topic  0
words related to the topics:
 0.055*"camera" + 0.035*"love" + 0.033*"buy" + 0.027*"like" + 0.024*"canon" + 0.018*"daughter" + 0.017*"get" + 0.016*"one" + 0.016*"purchase" + 0.015*"use"
words' topics: [(7, [0, 1]), (33, [0]), (68, [0, 1]), (78, [1, 2]), (81, 

...

words related to the topics:
 0.070*"camera" + 0.069*"great" + 0.050*"easy" + 0.044*"good" + 0.032*"use" + 0.026*"quality" + 0.026*"recommend" + 0.025*"lens" + 0.020*"price" + 0.020*"would"
words' topics: [(0, [2, 1, 0]), (29, [2, 1]), (78, [2, 1]), (85, [2, 1])]
-------------- 

review: 3239 

review topics: [(0, 0.3261107939407353), (1, 0.06359849353143501), (2, 0.6102907125278296)]
review topics(highest probs): Topic  2
words related to the topics:
 0.070*"camera" + 0.069*"great" + 0.050*"easy" + 0.044*"good" + 0.032*"use" + 0.026*"quality" + 0.026*"recommend" + 0.025*"lens" + 0.020*"price" + 0.020*"would"
words' topics: [(60, [2, 0]), (85, [2, 1]), (90, [2, 1]), (131, [0, 2]), (168, [0]), (379, [2, 0])]
-------------- 

review: 3240 

review topics: [(0, 0.07420097417436548), (1, 0.8549057167475619), (2, 0.0708933090780727)]
review topics(highest probs): Topic  1
words related to the topics:
 0.079*"camera" + 0.078*"great" + 0.073*"take" + 0.051*"picture" + 0.043*"love" + 0.035*"ea


review topics: [(0, 0.430951345691343), (1, 0.060531132761767394), (2, 0.5085175215468897)]
review topics(highest probs): Topic  2
words related to the topics:
 0.070*"camera" + 0.069*"great" + 0.050*"easy" + 0.044*"good" + 0.032*"use" + 0.026*"quality" + 0.026*"recommend" + 0.025*"lens" + 0.020*"price" + 0.020*"would"
words' topics: [(29, [2, 0, 1]), (168, [0]), (231, [2]), (269, [0, 1]), (436, [2])]
-------------- 

review: 3293 

review topics: [(0, 0.08848030612556868), (1, 0.09158392388278308), (2, 0.8199357699916483)]
review topics(highest probs): Topic  2
words related to the topics:
 0.070*"camera" + 0.069*"great" + 0.050*"easy" + 0.044*"good" + 0.032*"use" + 0.026*"quality" + 0.026*"recommend" + 0.025*"lens" + 0.020*"price" + 0.020*"would"
words' topics: [(0, [2, 1, 0]), (112, [2]), (350, [2])]
-------------- 

review: 3294 

review topics: [(0, 0.5655288739159577), (1, 0.1077444175704542), (2, 0.32672670851358815)]
review topics(highest probs): Topic  0
words related to the 

review: 3338 

review topics: [(0, 0.059139444779297314), (1, 0.8614898045607892), (2, 0.07937075065991354)]
review topics(highest probs): Topic  1
words related to the topics:
 0.079*"camera" + 0.078*"great" + 0.073*"take" + 0.051*"picture" + 0.043*"love" + 0.035*"easy" + 0.032*"use" + 0.020*"learn" + 0.018*"get" + 0.016*"buy"
words' topics: [(0, [1, 2]), (62, [1, 2]), (222, [1, 0]), (326, [1]), (1411, [1])]
-------------- 

review: 3339 

review topics: [(0, 0.04979935358274207), (1, 0.897376928296497), (2, 0.052823718120761066)]
review topics(highest probs): Topic  1
words related to the topics:
 0.079*"camera" + 0.078*"great" + 0.073*"take" + 0.051*"picture" + 0.043*"love" + 0.035*"easy" + 0.032*"use" + 0.020*"learn" + 0.018*"get" + 0.016*"buy"
words' topics: [(29, [1, 2]), (78, [1, 2]), (119, [1]), (135, [1]), (200, [1]), (913, [1])]
-------------- 

review: 3340 

review topics: [(0, 0.058443962110801446), (1, 0.061922856632333705), (2, 0.8796331812568648)]
review topics(highest 

review: 3379 

review topics: [(0, 0.04958432846580762), (1, 0.057995990577804864), (2, 0.8924196809563875)]
review topics(highest probs): Topic  2
words related to the topics:
 0.070*"camera" + 0.069*"great" + 0.050*"easy" + 0.044*"good" + 0.032*"use" + 0.026*"quality" + 0.026*"recommend" + 0.025*"lens" + 0.020*"price" + 0.020*"would"
words' topics: [(0, [2, 1, 0]), (85, [2, 1]), (163, [2]), (457, [2])]
-------------- 

review: 3380 

review topics: [(0, 0.06110279619763078), (1, 0.5913532853334876), (2, 0.3475439184688816)]
review topics(highest probs): Topic  1
words related to the topics:
 0.079*"camera" + 0.078*"great" + 0.073*"take" + 0.051*"picture" + 0.043*"love" + 0.035*"easy" + 0.032*"use" + 0.020*"learn" + 0.018*"get" + 0.016*"buy"
words' topics: [(17, [1, 2, 0]), (62, [2, 1]), (81, [1, 2, 0]), (106, [1]), (210, [2, 1])]
-------------- 

review: 3381 

review topics: [(0, 0.20704289178161223), (1, 0.6262863301615598), (2, 0.166670778056828)]
review topics(highest probs): Top

words related to the topics:
 0.070*"camera" + 0.069*"great" + 0.050*"easy" + 0.044*"good" + 0.032*"use" + 0.026*"quality" + 0.026*"recommend" + 0.025*"lens" + 0.020*"price" + 0.020*"would"
words' topics: [(0, [1, 2]), (78, [2, 1]), (85, [1, 2]), (178, [1, 2]), (469, [2, 0])]
-------------- 

review: 3446 

review topics: [(0, 0.8816560756480265), (1, 0.06278211713014188), (2, 0.05556180722183163)]
review topics(highest probs): Topic  0
words related to the topics:
 0.055*"camera" + 0.035*"love" + 0.033*"buy" + 0.027*"like" + 0.024*"canon" + 0.018*"daughter" + 0.017*"get" + 0.016*"one" + 0.016*"purchase" + 0.015*"use"
words' topics: [(33, [0]), (271, [0]), (351, [0, 1]), (580, [0]), (638, [0, 1])]
-------------- 

review: 3447 

review topics: [(0, 0.44393332656870665), (1, 0.11482175314519638), (2, 0.4412449202860969)]
review topics(highest probs): Topic  0
words related to the topics:
 0.055*"camera" + 0.035*"love" + 0.033*"buy" + 0.027*"like" + 0.024*"canon" + 0.018*"daughter" + 0.0

In [133]:
print("Unverified Topics:")
for idx_unverified, topic_unverified in lda_model_unverified.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_unverified, topic_unverified))
print("\n")
print("Verified Topics:")
for idx_verified, topic_verified in lda_model_verified.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Unverified Topics:
Topic: 0 
Words: 0.042*"camera" + 0.024*"great" + 0.024*"lens" + 0.020*"get" + 0.020*"use" + 0.018*"easy" + 0.018*"well" + 0.018*"quality" + 0.017*"take" + 0.016*"canon"
Topic: 1 
Words: 0.024*"take" + 0.022*"camera" + 0.022*"gift" + 0.016*"picture" + 0.015*"use" + 0.015*"want" + 0.015*"upgrade" + 0.015*"advantage" + 0.015*"card" + 0.009*"good"
Topic: 2 
Words: 0.039*"camera" + 0.036*"take" + 0.022*"picture" + 0.018*"love" + 0.018*"use" + 0.018*"want" + 0.015*"quality" + 0.015*"get" + 0.015*"come" + 0.011*"buy"


Verified Topics:
Topic: 0 
Words: 0.055*"camera" + 0.035*"love" + 0.033*"buy" + 0.027*"like" + 0.024*"canon" + 0.018*"daughter" + 0.017*"get" + 0.016*"one" + 0.016*"purchase" + 0.015*"use"
Topic: 1 
Words: 0.079*"camera" + 0.078*"great" + 0.073*"take" + 0.051*"picture" + 0.043*"love" + 0.035*"easy" + 0.032*"use" + 0.020*"learn" + 0.018*"get" + 0.016*"buy"
Topic: 2 
Words: 0.070*"camera" + 0.069*"great" + 0.050*"easy" + 0.044*"good" + 0.032*"use" + 0.026*"qua

__As we can see the contents between unverified and verified topics, both unverified and verified customers' put high values on this product. And a slightly different between these two is that unverified customers talk about the functions of the product as the words, "take", "lens" and "picture" have relatively higher probability than other words do; on the other hand, verified customers tend to praise the product since the words, "love", "great" and "easy" have relatively higher probability than other words do.__